Plan for steps in this processing code
1. Define a base grid
2. Load all the tif files - know how
4. Load the shp files - know how
5. Project the shp files to tif - at the resolution / bounds, etc with the base grid - know how
6. Generate the train deposit / occurence tif files - know how
6. Unify all the tif data - know how

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import rasterio
import rasterio.mask
from s2sphere import CellId
from shapely import Point
import pandas as pd

import multiprocessing as mp
from functools import partial

import utilities as utils

RAW_DATA_DIR = "data/LAWLEY22-RAW/geophysics/"
DERIV_DATA_DIR = "data/LAWLEY22-DERIV/geophysics/"

In [ ]:
tifs, shps = utils.get_input_var_files("Australia")

Loads the raster data

In [ ]:
rasters = utils.load_rasters(tifs, rasters_path=RAW_DATA_DIR, verbosity=1)

Loads rasters of the vector data if available; otherwise generates them

In [ ]:
try:
    rasters += utils.load_rasters(shps, rasters_path=RAW_DATA_DIR, verbosity=1)
except rasterio.RasterioIOError:
    base_raster = rasters[-1] # defaults to intermediate resolution raster
    vectors = utils.load_vectors(shps, vectors_path=RAW_DATA_DIR, verbosity=0)
    pbar = tqdm(zip(shps, vectors))
    for shp, vector in pbar:
        pbar.set_description(f"Processing {shp}")
        utils.proximity_raster_of_vector_points(base_raster, shp, vector)
    rasters += utils.load_rasters(shps, rasters_path=RAW_DATA_DIR, verbosity=1)

Loads the base grid for all data if available; otherwise generates it

In [ ]:
grid_cell_file = f"{DERIV_DATA_DIR}s2_grid_aus.npy"
try:
    grid_cell_ids = np.load(grid_cell_file)
except OSError:
    grid_bounds = utils.compute_bounds(rasters, verbosity=1)
    # gets all s2 cells
    grid_cells = utils.region_of_cellids(grid_bounds, s2_level=12)
    # filters s2 cells over water
    ocean = utils.load_vector("ne_10m_ocean","data/ocean/",verbosity=0)
    grid_cells = [grid_cell for grid_cell in grid_cells if not ocean["geometry"][0].intersects(Point(utils.s2_cell_center(grid_cell)))]
    # store remaining cell ids
    grid_cell_ids = [grid_cell.id() for grid_cell in grid_cells]
    np.save(grid_cell_file, np.asarray(grid_cell_ids, dtype=np.uint64))

Initialize the datacube

In [ ]:
datacube = utils.init_datacube({"s2_cell_id": grid_cell_ids}, ["s2_cell_center", "s2_cell_poly"] + tifs + shps, verbosity=1)

Populate the datacube using as many process as available CPUs

In [ ]:
def process_datacube(row, cols):
    s2cell = CellId(int(row[1]["s2_cell_id"]))
    row[1]["s2_cell_center"] = utils.s2_cell_center(s2cell)
    row[1]["s2_cell_poly"] = utils.s2_cell_polygon(s2cell)
    for col, raster in zip(cols, utils.load_rasters(cols)):
        try:
            masked, _ = rasterio.mask.mask(raster, [row[1]["s2_cell_poly"]], crop=True)
            if (raster.nodata == masked).all(): continue
            row[1][col] = np.mean(masked[raster.nodata != masked])
            row[1]["mask"] = True
        except ValueError:
            continue
    return row[1]

# process_datacube_multi = partial(process_datacube, cols=tifs + shps, rass=rasters)
process_datacube_multi = partial(process_datacube, cols=tifs + shps)

# set up multiprocessing pool
print(f"Number of threads populating datacube - {mp.cpu_count()}")
pool = mp.Pool(mp.cpu_count())

# apply function to DataFrame in parallel
results = []
for result in tqdm(pool.imap(process_datacube_multi, datacube.iterrows()), total=datacube.shape[0]):
    results.append(result)

# merge results back together
datacube = pd.DataFrame.from_records(results)

Store the datacube for future use

In [ ]:
datacube.to_csv(f"{DERIV_DATA_DIR}datacube.csv")

In [ ]:
datacube.info()